In [ ]:
# import modules
import os
import asyncio
import nest_asyncio

import season1.utility as util
import data_utility
import async_llm

from season1.prompt_generator import generate_prompt
from season1.preprocessor import preprocess

In [ ]:
# functions
def generate_examples(example_list, target_params):
    examples = "\n"
    for idx, data in enumerate(example_list):       
        examples += "Example " + str(idx + 1) + ":\nMap:\n"
        examples += data["map"]
        examples += "Parameters:\n"
        for param in target_params:
            examples += "- " + param + ": " + str(data["params"][param]) + "\n"
        examples += "\n"
    return examples

In [ ]:
# setting
target_params = ["map_size", "room_count", "enemy_count", "treasure_count", "exploration", "winding_path"] # Select one, two, or six from map_size, room_count, enemy_count, treasure_count, exploraion, winding_path
recursion_count = 5

dir_name = ""
for param in target_params:
    dir_name += param
    dir_name += "_"
dir_name += str(recursion_count)

result_dir = os.path.join("..", "data", "result", dir_name) 
map_data_path = os.path.join("..", "data", "DemoMapDataset.json")
target_param_path = os.path.join("..", "data", "TargetParameterDataset.json")
base_prompt_dir = os.path.join("..", "src", "base_prompt")

In [ ]:
# load dataset
map_dataset = util.read_json_file(map_data_path)
param_dataset = util.read_json_file(target_param_path)
param_dataset = param_dataset["param_list"]

In [ ]:
# create preparation prompt
preparation_templete = util.read_text_file(os.path.join(base_prompt_dir, "PreparationPhaseTemplete.txt"))

map_description = util.read_text_file(os.path.join(base_prompt_dir, "MapDescription.txt"))
preparation_templete = preparation_templete.replace("{MapDescription}", map_description)

preparation_prompts = []
for param in target_params:
    param_description = util.read_text_file(os.path.join(base_prompt_dir, param + ".txt"))
    
    example_list = data_utility.get_demos_from_map_dataset(map_dataset, [param])
    examples = generate_examples(example_list, target_params)

    parameters = util.read_text_file(os.path.join(base_prompt_dir, "ParameterTemplete.txt"))
    parameters = parameters.replace("{ParameterDescription}", param_description)
    parameters = parameters.replace("{Examples}", examples)
    
    preparation_prompts.append(preparation_templete.replace("{Parameters}", parameters))

In [ ]:
# preparation phase
system_prompt = """
You are an expert in analyzing and modifying ASCII dungeon maps based on given parameters. Your task is to help users understand how various parameters affect the layout of the map and to provide step-by-step instructions on how to modify the map to achieve a target parameter value. Ensure that your explanations are clear and structured, and that you provide logical reasoning for each modification. You will also describe both how to increase and decrease specific parameter values.
"""

nest_asyncio.apply()

preparation_outputs = asyncio.run(async_llm.generate_unstructured_datas(system_prompt, preparation_prompts))

In [ ]:
# create modification prompt templete
modification_templete = util.read_text_file(os.path.join(base_prompt_dir, "ModificationPhaseTemplete.txt"))
modification_templete = modification_templete.replace("{MapDescription}", map_description)

parameters = util.read_text_file(os.path.join(base_prompt_dir, "ParameterTemplete.txt"))

param_description = ""
for param in target_params:
    param_description += util.read_text_file(os.path.join(base_prompt_dir, param + ".txt")) + '\n'

parameters = parameters.replace("{ParameterDescription}", param_description)
parameters = parameters.replace("{Examples}", examples)

modification_templete = modification_templete.replace("{Parameters}", parameters)

preparation_output = "\n".join(preparation_outputs)

modification_templete = modification_templete.replace("{PreparationOutput}", preparation_output)

In [ ]:
# create initial prompt
user_prompts = []
used_examples = []
used_params = []
for params in param_dataset:
    example_list = data_utility.get_demos_from_map_dataset(map_dataset, target_params)
    examples = generate_examples(example_list, target_params)
    
    system_prompt, user = generate_prompt(examples, params, "AutoCOT2")
    user_prompts.append(user)
    used_examples.append(example_list)
    used_params.append(params)

In [ ]:
# initial creation phase
#initial_outputs = asyncio.run(async_llm.generate_unstructured_datas(system_prompt, user_prompts))
initial_outputs = [""] * 1000

In [ ]:
# preprocess initial outputs
preprocessed_outputs = list(map(preprocess, initial_outputs))

In [ ]:
# make block
data_blocks = []
for i in range(1000):
    data_blocks.append({
        "params": used_params[i],
        "map": preprocessed_outputs[i],
        "examples": used_examples[i]
    })